In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.python.keras import backend as K
import tensorflow as tf
import collections
import numpy as np
from openpyxl import load_workbook
from openpyxl import Workbook
from pandas import ExcelWriter
import pandas.io.formats.excel
#from funciones import ErrorAuno,ErrorAcero
from funciones import ErrorAuno,ErrorAcero,TensorUpdatePosicionInicial,TensorUpdateCiclo
from tensorflow.keras.layers import (Activation, AveragePooling2D, BatchNormalization, Cropping2D,
                                     Concatenate, Conv1D, Conv2D, Dense, DepthwiseConv2D, Dropout,
                                     Embedding, Flatten, GlobalAveragePooling2D, Lambda, MaxPool2D,
                                     MaxPooling1D, ReLU, Reshape, ZeroPadding2D)

In [7]:
##### Funciones para trabajar cada caso según el tipo de error si el mismo está dado por la escritura de un 0 cuando
##### debía ser un 1(ErrorAcero) o viceversa. Se ha desarrollado un algoritmo que permite colocar luego del primer bit con
####error encontrado en el resto de las posiciones a 0 o a unos : Se analizan los valores comenzando por el BMS(Bit más
#signif)comparando el valor con el error y el valor que se debe escribir correcto donde haya un cambio de valor se analiza
#### si el cambio es de un 0 x 1 se coloca el resto de los bits a 0, y luego se inyectan los demás errores y si el cambio
#### es de un 1 x 0 se colocan los bits siguientes a 1 y se mantienen los errores restantes haciendo un Bitwise con los
#valores  generados por errores estaticos a 1 o a 0

In [9]:
# Conversión del tensor generado por la red a naturales: para ello se multiplica el número generado por la red que es un 
#un número real x 2**bit de la parte fraccionaria colocando el valor en un entero de 32 bits(de esta forma se convierten a 
# enteros) luego para llevarlo a naturales(a los valores negativos se les suma 2**al tamaño de la palabra que en este caso
# es de 16 bits)

##Prueba para entrar a errores por 0

In [2]:

tensor_original=tf.convert_to_tensor([40167,61311,64511,22,17,37,30,38 ])
print('tensor_original',tensor_original)



tensor_original tf.Tensor([40167 61311 64511    22    17    37    30    38], shape=(8,), dtype=int32)


In [3]:
static_Error0_1= ([7927,528],[28671,0],[57343,1024],[65535,12],[65518,4],[65535,32768],[49087,0] ,[49087,0])
faults=tf.convert_to_tensor(static_Error0_1)
tensor  = tf.bitwise.bitwise_and(tensor_original,faults[:,0])
print('faults[:,0]',faults[:,0])
tensor_with_error  = tf.bitwise.bitwise_or(tensor,faults[:,1])
tensor_with_error

faults[:,0] tf.Tensor([ 7927 28671 57343 65535 65518 65535 49087 49087], shape=(8,), dtype=int32)


<tf.Tensor: shape=(8,), dtype=int32, numpy=array([ 7927, 28543, 57343,    30,     4, 32805,    30,    38])>

### otra prueba de datos

In [4]:
tensor_original=tf.convert_to_tensor([38,    38, 32769 ])
static_Error0_1=([ 49087,16384] ,[49087,64],[65471,514])
faults=tf.convert_to_tensor(static_Error0_1)
tensor  = tf.bitwise.bitwise_and(tensor_original,faults[:,0])
print('faults[:,0]',faults[:,0])
tensor_with_error  = tf.bitwise.bitwise_or(tensor,faults[:,1])
tensor_with_error

faults[:,0] tf.Tensor([49087 49087 65471], shape=(3,), dtype=int32)


<tf.Tensor: shape=(3,), dtype=int32, numpy=array([16422,   102, 33283])>

## Declaración de variables y preparación de los datos que entran al ciclo

In [ ]:
#values_to_run: Valores (v_b) donde no se han encontrado errores para la máscara analizada y deben entrar al ciclo
#mask:1000000000000000
#v_m:Valores afectado por los errores son los que luego de inyectar errores son diferentes a los originales
#v_b: Valores correspondientes a los de arriba es decir los que se deberia escribir en memoria
#mask_0: matriz de control de los cambios que voy realizando se utiliza al inicio cuando entra al ciclo
#valores_afectados:Matriz de control en un inicio me da en true los valores que difieren entre el tensor_original y el tensor con error
#
#valores_afec_static_0_Error:Errores en 0 que propician cambios en el tensor original
#valores_afec_static_1_Error::Errores en 1 que propician cambios en el tensor original
#vb_and_mask:Matriz de true an false resultado de btw_and entre los valores originales y la máscara
#vm_and_mask:Matriz de true an false resultado de btw_and entre los valores con error y la máscara
## se hace con el fin de determinar donde no soniguales sus resultados porque en este caso hay un error en los elementos analizados    
#tensor_act: En un inicio se iguala convenientemente al tensor original y luego es el que se va podicifando con
# los valores nuevos    


In [ ]:
##Todo este código que viene lo convierto en una función llamada FuncionRoundTensor para llamarla dentro del Net.py en una 
##función que cree (valueNeares) muy oarecida a Aging pero haciendo esta llamda y luego continuar con el resto.

In [4]:
#Se crea un tensor variable donde se guardarán los valores en cada ciclo que aún no se han encontrado la posición con error
#y hay que continuar analizando con mask>>1
# se cra un tensor con el valos de la máscara inicial la cual es 1000000000000000
values_to_run=tf.Variable(0)
mask=tf.convert_to_tensor(32768)

#crea un tensor de (T and F),donde en T están los valores afectados por los errores inyectados
# esta matriz es la que controla si ya no hay cambios pendientes porque todos los valores han sido analizados y
#detiene el ciclo para no iterar innecesariamente

valores_afectados=tf.math.not_equal(tensor_original,tensor_with_error)
print('valores_afectados',valores_afectados)  

# Se extraen los valores que fueron afectados por los errores
v_m=tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True))
print('valores afectados por los errores' ,v_m)

#valores que se deberian escribir
v_b=tf.gather_nd(tensor_original,tf.where(valores_afectados==True))
print('valores que se deberian escribir' ,v_b)
#a=np.size(v_m)
# Creo una máscara de 0 del con la cantidad de errores afectados para ir llevando el contrl de los cambios 
#mask_0=tf.convert_to_tensor([0]*a)
mask_0=tf.constant(0)
print('mask_0',mask_0)

# inicializo los valores de comienzo y fin del ciclo teniendo en cuenta que pueden haber errores hasta la posición 15 de
# una palabra
i=tf.constant(0)
out=tf.constant(16)
print(' out inicial',(out))
print('i inicial', i)


#matriz_control(esta la cree pero al final no la uso luego que verifique todo puede ser eliminada sino hace falta)
#matriz_control=tf.equal(tensor_original,tensor_with_error)
matriz_control_cambios=tf.equal(v_b,v_m)
print('matriz_control_cambios',matriz_control_cambios)

# Tensores con los valores de errores en 0 y 1 que propiciaron cambios en el tensor original
valores_afec_static_0_Error=tf.gather_nd(faults[:,0], tf.where(tf.math.not_equal(tensor_original,tensor_with_error)))
valores_afec_static_1_Error=tf.gather_nd(faults[:,1], tf.where(tf.math.not_equal(tensor_original,tensor_with_error)))
print('valores_afec_static_0_Error',valores_afec_static_0_Error)
print('valores_afec_static_1_Error',valores_afec_static_1_Error)

vb_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_original,tf.where(valores_afectados==True)),mask)
print('vb_and_mask',vb_and_mask)
vm_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True)),mask)
vm_and_mask
##### 'aquí actualizo el tensor con los nuevos valores de igual a mask'
print('vm_and_mask',vm_and_mask)
#tf.gather_nd(v_b,tf.where(tf.equal(vb_and_mask,vb_and_mask)):
#a=np.size(v_m)

## la primera vez tensor_actu= tensor original creo esta variable para ir actualizando el tensor con los nuevos valores 
#generados con la técnica del redeondeo

tensor_act=tensor_original 

valores_afectados tf.Tensor([ True  True  True  True  True  True False False], shape=(8,), dtype=bool)
valores afectados por los errores tf.Tensor([ 7927 28543 57343    30     4 32805], shape=(6,), dtype=int32)
valores que se deberian escribir tf.Tensor([40167 61311 64511    22    17    37], shape=(6,), dtype=int32)
mask_0 tf.Tensor(0, shape=(), dtype=int32)
 out inicial tf.Tensor(16, shape=(), dtype=int32)
i inicial tf.Tensor(0, shape=(), dtype=int32)
matriz_control_cambios tf.Tensor([False False False False False False], shape=(6,), dtype=bool)
valores_afec_static_0_Error tf.Tensor([ 7927 28671 57343 65535 65518 65535], shape=(6,), dtype=int32)
valores_afec_static_1_Error tf.Tensor([  528     0  1024    12     4 32768], shape=(6,), dtype=int32)
vb_and_mask tf.Tensor([32768 32768 32768     0     0     0], shape=(6,), dtype=int32)
vm_and_mask tf.Tensor([    0     0 32768     0     0 32768], shape=(6,), dtype=int32)


In [14]:
## depurar para colocar en o donde hay problema en el primer bit y luego que entre al ciclo esto va a ser una función 
#a:Matriz de True and False donde True son las posiciíones donde vb_and_mask y vm_and_mask son diferentes
#por tanto existe un error en la posición más significativa 
#b:Matriz de True and False donde True  es la posiciíon dende se cumple que (v_m>v_b ) 
#c: Indice donde la matriz a y b son verdaderas por tanto el error esta dado por un 1 
#values_to_change:los valores de v_b en las posiciones c que han se ser enviados a la función ErroraUno para su transformación
#val_afec_0_Error_to_change:Valores que corresponden los el error a 0 
#val_afec_1_Error_to_change:valores que corresponden con elerror a 1  Creo que ya no son necesarias 
#index: devuelve el valor de c en una variable numpy
#val:devuelve una lista de elementos que corresponden a los indices de valores afectados ==True que serán los indices a transformar
#error_a_0: Matriz de True and False donde True  es la posiciíon dende se cumple que (v_b>v_m )
#index_values_error_a_0:Indice donde la matriz a y error_a_0 que son verdaderas por tanto el error esta dado por un 0 
#val_0=devuelve una lista de elementos que corresponden a los indices de valores afectados ==True que serán los indices a transformar
#tensor_act: devuelve el tensor ya transormado con los elementos en las posiciones correspondientes donde se han encontrado los casos analizados


In [24]:
#def ErrorPosicionInicial(v_m,v_b):
#    b=tf.math.greater(v_m,v_b ) # El error es a 1 y aplico la variante con_error_a_1
#    print('b Vm>Vb',b)
#    c=tf.where(tf.logical_and(a,b)==True)
#    print('c',c)
#    print('tamaño de c',tf.size(c))
#    print(' c Indices de los valores a transformar a 0',c)
#    values_to_change=tf.gather_nd(v_b,c)
#    print('mask_0 desdes update', mask_0)
#    index=c.numpy()
#    print('cnumpy',index)
#    print('valores_afectados',valores_afectados)
#    val=tf.gather_nd(tf.where(valores_afectados),index)## tomo los indices del tensor original qu ese cambiaran
#    print('val',val)
#    #mask_0=tf.add(mask_0,tf.size(val))
#    return(val)

In [ ]:
##  Actualiza el tensor  colocando los valores a 0 en el caso que el error esté en la posición inicial es decir 
#antes de entrar al ciclo, da igual el tipo de error que sea porque esto implica un erroer en el signo

In [13]:
def TensorUpdatePosicionInicial(tensor_act,val,valores_afectados,mask_0,tensor_with_error,faults): 
    print('Dentro de TensorUpdatePosicionInicial')
    tensor_act = tf.tensor_scatter_nd_update(tensor_act, val, tf.convert_to_tensor([0]*np.size(val) ))
    valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val,tf.convert_to_tensor([False]*np.size(val)))
    print('valores_afectados dentro de error 0 en primera posición',valores_afectados )
    mask_0=tf.add(mask_0,tf.size(val))
    print('tensor_act funciion a 0',tensor_act)
    print('mask_0',mask_0)
    print('valores_afectados',valores_afectados)
    print('tensor_with_error',tensor_with_error)
    #tensor_with_error_run= tf.gather_nd(tensor_with_error,tf.where(valores_afectados ==True))   
    v_b=tf.gather_nd(tensor_act,tf.where(valores_afectados==True))
    print('v_b',v_b)
    v_m=tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True) )
    print('v_m',v_m)
    valores_afec_static_0_Error=tf.gather_nd(faults[:,0],tf.where(valores_afectados==True))
    print('valores_afec_static_0_Error', valores_afec_static_0_Error)
    valores_afec_static_1_Error=tf.gather_nd(faults[:,1],tf.where(valores_afectados==True))
    print('valores_afec_static_1_Error', valores_afec_static_0_Error)
    return tensor_act,valores_afectados,mask_0,v_b,v_m,valores_afec_static_0_Error,valores_afec_static_1_Error

In [ ]:
## Actualiza el tensor una vez entrado al ciclo y en dependencia del valor de error va a la función correspondiente
## ya sea que error ha sido el cambio de un 0 por un 1 o viceversa(son algoritmos diferentes para cada caso)

In [14]:
def TensorUpdateCiclo(error,v_b,c,valores_afec_static_0_Error,valores_afec_static_1_Error,mask,tensor_act,valores_afectados,mask_0):       
    values_to_change=tf.gather_nd(v_b,c)
    val_afec_0_Error_to_change=tf.gather_nd(valores_afec_static_0_Error,c)
    print('val_afec_0_Error_to_change',val_afec_0_Error_to_change)
    val_afec_1_Error_to_change=tf.gather_nd(valores_afec_static_1_Error,c)
    print('val_afec_1_Error_to_change',val_afec_1_Error_to_change)
    print('values_to_change',values_to_change)
    if error==1:
        tensor_round=ErrorAuno(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change)
        print('tensor_round',tensor_round)
    if error==0:
        tensor_round=ErrorAcero(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change)
        print('tensor_round',tensor_round)
    index=c.numpy()
    print('indice a camniar tensor act',index)
    print('valores_afectados',valores_afectados)
    val=tf.gather_nd(tf.where(valores_afectados),index)## tomo los indices del tensor original qu ese cambiaran
    print('val',val)
    tensor_act = tf.tensor_scatter_nd_update(tensor_act, val, tensor_round )
    valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val,tf.convert_to_tensor([False]*np.size(val)))
    mask_0=tf.add(mask_0,tf.size(val))
    print('mask_0 desdes update', mask_0)
    print('tensor tensor_act',tensor_act)
    print('valores_afectados update',valores_afectados) 
    #del tensor_round
    print('tensor_round',tensor_round)
    return tensor_act,valores_afectados,mask_0
    

In [5]:
a=tf.math.not_equal(vb_and_mask,vm_and_mask)
print('estoy calculando a y definiendo xq esta dado el error')
print('a vb_and_mask!=vm_and_mask',a)
error_a_1=tf.math.greater(v_m,v_b ) # El error es a 1 y aplico la variante con_error_a_1
print('b Vm>Vb',error_a_1)
index_values_error_a_1=tf.where(tf.logical_and(a,error_a_1)==True)
print('c',index_values_error_a_1)
print('tamaño de c',tf.size(index_values_error_a_1))
print(' c Indices de los valores a transformar a 0',index_values_error_a_1)
values_to_change=tf.gather_nd(v_b,index_values_error_a_1)
print('mask_0 desdes update', mask_0)
#index=index_values_error_a_1.numpy() Esto da error por tanto uso la variante de abajo
index = tf.experimental.numpy.array(index_values_error_a_1)#para obtener el numero como índices
print('cnumpy',index)
print('valores_afectados',valores_afectados)
val=tf.gather_nd(tf.where(valores_afectados),index)## tomo los indices del tensor original qu ese cambiaran
print('val',val)



#### agraegado para si se encuentra error a 0 en la posición más significativa  esto 
error_a_0=tf.math.greater(v_b,v_m)
index_values_error_a_0=tf.where(tf.logical_and(a,error_a_0)==True)
print('index_values_error_a_0',index_values_error_a_0)
index_0=index_values_error_a_0.numpy()
print('cnumpy',index_0)
print('valores_afectados',valores_afectados)
val_0=tf.gather_nd(tf.where(valores_afectados),index_0)## tomo los indices del tensor original que se cambiaran

## Esta es la condición que da error en l proceso de inferencia cuando corro algún experimento real
# Aquí lo que hago es actualizar el tensor donde se encontraron valores con errores en el signo lo que se hace es 
#llamar a una función que los coloca en 0 y actualiza los valores que deben entrar alciclo
#val_0:índices del tensor original donde el cambio fue de un 1 por un 0 en el signo
#val:índices del tensor original donde el cambio fue de un 0 por un 1  en el signo

## entra a estos  if cuando hay error a 1 0 a cero en la posición más significativa    
if np.size(val)> 0: 
    tensor_act,valores_afectados,mask_0,v_b,v_m,valores_afec_static_0_Error,valores_afec_static_1_Error=TensorUpdatePosicionInicial(tensor_act,val,valores_afectados,mask_0,tensor_with_error,faults)

if np.size(val_0)>0:
    tensor_act,valores_afectados,mask_0,v_b,v_m,valores_afec_static_0_Error,valores_afec_static_1_Error=TensorUpdatePosicionInicial(tensor_act,val_0,valores_afectados,mask_0,tensor_with_error,faults)
        
    
print('tensor_act',tensor_act)
print('valores_afectados',valores_afectados)
print('tensor_with_error',tensor_with_error)
print('v_b',v_b)
print('v_m',v_m)
print('valores_afec_static_0_Error', valores_afec_static_0_Error)
print('valores_afec_static_1_Error',valores_afec_static_1_Error)
mask=tf.bitwise.right_shift(mask,1)

estoy calculando a y definiendo xq esta dado el error
a vb_and_mask!=vm_and_mask tf.Tensor([ True  True False False False  True], shape=(6,), dtype=bool)
b Vm>Vb tf.Tensor([False False False  True False  True], shape=(6,), dtype=bool)
c tf.Tensor([[5]], shape=(1, 1), dtype=int64)
tamaño de c tf.Tensor(1, shape=(), dtype=int32)
 c Indices de los valores a transformar a 0 tf.Tensor([[5]], shape=(1, 1), dtype=int64)
mask_0 desdes update tf.Tensor(0, shape=(), dtype=int32)
cnumpy tf.Tensor([[5]], shape=(1, 1), dtype=int64)
valores_afectados tf.Tensor([ True  True  True  True  True  True False False], shape=(8,), dtype=bool)
val tf.Tensor([[5]], shape=(1, 1), dtype=int64)
index_values_error_a_0 tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64)
cnumpy [[0]
 [1]]
valores_afectados tf.Tensor([ True  True  True  True  True  True False False], shape=(8,), dtype=bool)


TypeError: TensorUpdatePosicionInicial() missing 1 required positional argument: 'index'

In [ ]:
## prubas para rectificar que todo esta bien

In [ ]:
vb 40167,61311,64511
error_0: 7927,28671,57343
error_1: 528,0,1024
0.Antes de entrar al ciclo
1001110011100111 vb 40167
0001111011110111 vm 7927
0001111011110111 error_0 7927
0001111011110111 and
0000001000010000 error_1 528
0001111011110111 7927
Prueba
mask=32768     
n_mask=mask-1   0111111111111111
1001110011100111 vb
0111111111111111   n_mask
1111111111111111 or 65535
0001111011110111error_0 7927
0001111011110111 and  7927
0000001000010000 or error_1
0001111011110111 7927


1. Antes de entrar al ciclo
1110111101111111 vb 61311
0110111111111111 error_0 28671
0110111101111111 28543
0000000000000000 error_1 
0110111101111111 28543
Prueba
mask=32768     
n_mask=(mask-1) 0111111111111111
1110111101111111 vb
0111111111111111 n_mask
1111111111111111 or 65535
0110111111111111 and error_0 28671
0110111111111111
0000000000000000 error_1
0110111111111111 or  28671 

3. iteración i=1
1111101111111111 vb 64511
1101111111111111 v_m 57343
1101111111111111 error_0 57343
1101101111111111
0000010000000000 error_1 1024
1101111111111111 57343

Prueba
mask=8192 0010000000000000    
n_mask=(mask-1)0001111111111111
1111101111111111 vb 64511
0001111111111111  n_mask
1111111111111111 or n_mask
1101111111111111 error_0 57343
1101111111111111 and
0000010000000000 error_1 1024
1101111111111111  57343




1111101111111111v_b
1000000000000000
1000000000000000 vb_mask
1101111111111111- v_m
1000000000000000
1000000000000000 vm_mask
vb_mask!=vm_mask  false
v_m>v_b false



In [34]:
bin( 8191)

'0b1111111111111'

In [35]:
int('1101111111111111',2)

57343

## modificarlo porque en el caso donde hay 0 en la posición 1 no lo estaba contemplando

In [ ]:
#values_to_run: Valores donde no se ha encontrado error y deben seguir iterandose con ellos (v_b)
#values_to_run_with_error:Valores con errores corespondientes en posición con los de arriba que tb deben seguir iteransose
#valores_afec_static_0_Error_to_run:valores que porporcionan error en 0 correspondiente a los de arriba
#valores_afec_static_0_Error_to_run:valores que porporcionan error en 1 correspondiente a los de arriba
#values_to_change:Valores en los que se ha encontrado error con la máscara actual y se van a modificar (Ya no lo uso porque optimicé el código)

In [20]:
def while_body(i,out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error):
    print('dentro del body')
    print('i',i)
    print('out que llega al body', out)
    print('tensor_act',tensor_act)
    print('tensor_original',tensor_original)
    print('mask',mask)
    print('mask_0',mask_0)
    print('v_m',v_m)
    print('v_b',v_b)
    print('tensor_with_error',tensor_with_error)
    
        
    ## Este if lo cree por si hay cambios antes de llegar al ciclo por haber errores e la posición 15 si mask_0 ha tomado valor
    ## es porque hubo cambios y se analizan los valores que quedan pendientes con la máscara nueva
    # necesraio analizar luego de experimentar  si mask_0 llega con valores a 1 es porque se hicieron cambios y se deben
    #analizar los btw con los valores resttantes sino todo se mantiene con los valores originales de entrada.
    if tf.reduce_sum(mask_0)>0:
        print('entre en condicion mask_0>0')
        print('valores donde debo correr el mask')
        
        vb_and_mask=tf.bitwise.bitwise_and(v_b,mask)
        print('vb_and_mask',vb_and_mask)
        vm_and_mask=tf.bitwise.bitwise_and(v_m,mask)
        print('vm_and_mask',vm_and_mask)  
    else:
        
         ### Si no se encontraron errores antes de entrar al ciclo se realizan las operaciones correspondientes con
         ### la nueva máscara(0100000000000000)
         
        print('entre en el else')
        vb_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_original,tf.where(valores_afectados==True)),mask)
        print('vb_and_mask',vb_and_mask)
        vm_and_mask=tf.bitwise.bitwise_and(tf.gather_nd(tensor_with_error,tf.where(valores_afectados==True)),mask)
        print('vm_and_mask',vm_and_mask) 
        print('v_b',v_b)
        print('v_m',v_m)
        
    a=tf.math.not_equal(vb_and_mask,vm_and_mask)
    print('estoy calculando a y definiendo xq esta dado el error')
    print('a vb_and_mask!=vm_and_mask',a)
    values_to_run=tf.gather_nd(v_b,tf.where(a==False))
    values_to_run_with_error=tf.gather_nd(v_m,tf.where(a==False))
    valores_afec_static_0_Error_to_run=tf.gather_nd(valores_afec_static_0_Error,tf.where(a==False))
    print('valores_afec_static_0_Error_to_run',valores_afec_static_0_Error_to_run)
    valores_afec_static_1_Error_to_run=tf.gather_nd(valores_afec_static_1_Error,tf.where(a==False))
    print('valores_afec_static_1_Error_to_run',valores_afec_static_1_Error_to_run)
    print('values_to_run', values_to_run)
    print(' values_to_run_with_error', values_to_run_with_error)
    #valores_afectados_to_run= valores_afectados
    #u=tf.size(values_to_run)>0
    
    
    error_a_1=tf.math.greater(v_m,v_b ) # El error es a 1 y aplico la variante con_error_a_1
    print('b Vm>Vb',error_a_1)
    index_values_error_a_1=tf.where(tf.logical_and(a,error_a_1)==True)
    #c=tf.where(tf.equal(a,b))
    print('tamaño de c',tf.size(index_values_error_a_1))
    print(' c Indices de los valores a transformar con la variante error a 1',index_values_error_a_1)
    
    error_a_0=tf.math.greater(v_b,v_m)
    print('error_a_0',error_a_0)
    index_values_error_a_0=tf.where(tf.logical_and(a,error_a_0)==True)
    #index_values_error_a_0=tf.where(error_a_0==True)
    print('index_values_error_a_0',index_values_error_a_0)
    print('vb',v_b)
    
        
   
    if tf.size(index_values_error_a_1)>0:
        error=1
        tensor_act,valores_afectados,mask_0,val_afec_0_Error_to_change,val_afec_1_Error_to_change= TensorUpdateCiclo(error,v_b,index_values_error_a_1,valores_afec_static_0_Error,valores_afec_static_1_Error,mask,tensor_act,valores_afectados,mask_0)
        #values_to_change=tf.gather_nd(v_b,c)
        #val_afec_0_Error_to_change=tf.gather_nd(valores_afec_static_0_Error,c)
        #print('val_afec_0_Error_to_change',val_afec_0_Error_to_change)
        #val_afec_1_Error_to_change=tf.gather_nd(valores_afec_static_1_Error,c)
        #print('val_afec_1_Error_to_change',val_afec_1_Error_to_change)
        #print('values_to_change',values_to_change)
        #tensor_round=ErrorAuno(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change)
        #print('tensor_round',tensor_round)
        #index=c.numpy()
        #print('indice a camniar tensor act',index)
        #print('valores_afectados',valores_afectados)
        #val=tf.gather_nd(tf.where(valores_afectados),index)## tomo los indices del tensor original qu ese cambiaran
        #print('val',val)
        #tensor_act = tf.tensor_scatter_nd_update(tensor_act, val, tensor_round )
        #valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val,tf.convert_to_tensor([False]*np.size(val)))
        #mask_0=tf.add(mask_0,tf.size(val))
        print('mask_0 desdes update', mask_0)
        print('tensor tensor_act',tensor_act)
        print('valores_afectados update',valores_afectados)                                              
    
    
        
        
        
    
        
    ##no es el que esta en mask en esse momento, retorno new mask, new_mask_0, tensor_act
    if tf.size(index_values_error_a_0)>0: 
        error=0
        print('dentro detf.size(error_a_0)>0 ')
        tensor_act,valores_afectados,mask_0,val_afec_0_Error_to_change,val_afec_1_Error_to_change= TensorUpdateCiclo(error,v_b,index_values_error_a_0,valores_afec_static_0_Error,valores_afec_static_1_Error,mask,tensor_act,valores_afectados,mask_0)
        #out=tf.constant(tf.math.subtract(out,tf.size(error_a_0)))
        #print('out',out)
        #mask_0=tf.tensor_scatter_nd_update(mask_0,error_a_0,tf.convert_to_tensor([1]*np.size(error_a_0)))
        print()
        #out=tf.constant(tf.math.subtract(out,tf.reduce_sum(mask_0)))
        #values_to_change=tf.gather_nd(v_b,index_values_error_a_0)
        #val_afec_0_Error_to_change=tf.gather_nd(valores_afec_static_0_Error,index_values_error_a_0)
        #print('val_afec_0_Error_to_change',val_afec_0_Error_to_change)
        #val_afec_1_Error_to_change=tf.gather_nd(valores_afec_static_1_Error,index_values_error_a_0)
        #print('val_afec_1_Error_to_change',val_afec_1_Error_to_change)
        #print('values_to_change',values_to_change)
        #tensor_round=ErrorAcero(values_to_change,mask,val_afec_0_Error_to_change,val_afec_1_Error_to_change)
        #index_0=index_values_error_a_0.numpy()
        #print('index_0',index_0)
        #print('valores_afectados',valores_afectados)
        #val_0=tf.gather_nd(tf.where(valores_afectados),index_0)## tomo los indices del tensor original qu ese cambiaran
        #print('val_0',val_0)
        #
        #tensor_act = tf.tensor_scatter_nd_update(tensor_act, val_0, tensor_round )
        #valores_afectados=tf.tensor_scatter_nd_update(valores_afectados,val_0,tf.convert_to_tensor([False]*np.size(val_0)))
        ##print('tensor_round en el body',tensor_round)
        print('tensor tensor_act',tensor_act)
        print('valores_afectados update',valores_afectados)        
        #mask_0=tf.add(mask_0,tf.size(val_0))
        print('mask_0 desdes update', mask_0)
    
        
        
        
    if tf.size(values_to_run)>0:
        #si existen valores que aún no se han modificados cambio la máscara actualizo variables y continua el ciclo
        mask=tf.bitwise.right_shift(mask,1)
        print('dentro del if values to run',values_to_run)
        print('mask',mask)
        v_b=values_to_run
        v_m=values_to_run_with_error 
        valores_afec_static_0_Error=valores_afec_static_0_Error_to_run
        valores_afec_static_1_Error=valores_afec_static_1_Error_to_run
        print('v_b',v_b)
        print('v_m',v_m)
        print('valores_afec_static_0_Error',valores_afec_static_0_Error)
        print('valores_afec_static_1_Error',valores_afec_static_1_Error)
        
    
    
    if tf.experimental.numpy.any(valores_afectados)==False:
        # Cuando toda esta matriz sea false se detiene el ciclo aunque no haya iterado 15 veces para evitar iteraciones innecesarias
        # en caso que no hayan errores en las posiciones menos significativas
        #tensor=tf.bitwise.bitwise_and(tensor_act,val_afec_0_Error_to_change)
        #print('n_v_b_1', tensor)
        #tensor_round=tf.bitwise.bitwise_or(tensor,val_afec_1_Error_to_change)
        print('valores_afectados',valores_afectados)
        print('tensor_act',tensor_act)
        i=16
        
    print('i',i) 
    
    
    return (i+1),out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error ## retorno los valores que aun no se han modificado por ninguna d elas variantes puesto que el bit con error     
    
i,out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error= tf.while_loop(
    lambda i, *_ : tf.less(i,out),  # condición -> revisar cada bit
     while_body,  # aqui todo lo que tengas que hacer de comparar los vectores (que estos los puedes definir por fuera
                 # o pasarlos a la misma función de while_body y no modificarlos)
    (i,out,mask,mask_0,tensor_original,tensor_act,v_m,v_b,tensor_with_error,values_to_run,valores_afectados,valores_afec_static_0_Error,valores_afec_static_1_Error)  # valores iniciales
)   



dentro del body
i tf.Tensor(0, shape=(), dtype=int32)
out que llega al body tf.Tensor(16, shape=(), dtype=int32)
tensor_act tf.Tensor([    0     0 64511    22    17     0    30    38], shape=(8,), dtype=int32)
tensor_original tf.Tensor([40167 61311 64511    22    17    37    30    38], shape=(8,), dtype=int32)
mask tf.Tensor(16384, shape=(), dtype=int32)
mask_0 tf.Tensor(3, shape=(), dtype=int32)
v_m tf.Tensor([57343    30     4], shape=(3,), dtype=int32)
v_b tf.Tensor([64511    22    17], shape=(3,), dtype=int32)
tensor_with_error tf.Tensor([ 7927 28543 57343    30     4 32805    30    38], shape=(8,), dtype=int32)
entre en condicion mask_0>0
valores donde debo correr el mask
vb_and_mask tf.Tensor([16384     0     0], shape=(3,), dtype=int32)
vm_and_mask tf.Tensor([16384     0     0], shape=(3,), dtype=int32)
estoy calculando a y definiendo xq esta dado el error
a vb_and_mask!=vm_and_mask tf.Tensor([False False False], shape=(3,), dtype=bool)
valores_afec_static_0_Error_to_run tf.Tens

In [9]:
bin(37)

'0b100101'

In [ ]:
1000000000100101
0000000000100101

In [ ]:
#resultado que debe dar con el ejemplo más grande de arriba sin inyectar los errores  
 sin inyectar errores  : 0     0 65535    16    31     0    30    38  

In [ ]:
inyectando errores tensor_act tf.Tensor([    0     0 57343    28    14     0    30    38],
                                        
 sin inyectar errores                     0     0 65535    16    31     0    30    38                                       

In [ ]:
## luego inyecto los errores y listo 

In [10]:
tensor=tf.convert_to_tensor( [ 0,0, 65535, 16 ,   31,  0,  30  , 38] ) 
static_0_Error=tf.convert_to_tensor([7927,28671,57343,65535,65518,65535,49087 ,49087])
static_1_Error=tf.convert_to_tensor([528,0,1024,12,4,32768,0,0 ])
tensor=tf.bitwise.bitwise_and(tensor,static_0_Error)
tensor=tf.bitwise.bitwise_or(tensor,static_1_Error)
tensor

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([  528,     0, 57343,    28,    14, 32768,    30,    38])>